# Notebook 

Notebook de renforcement learning pour le TER LeekWars.


Membres du groupe : 

* Alexandre Cauty 
* Loic Schnuriger 
* Thomas Juliat
* Aissaoui Lucas

## Import 
Improt nécéssaire au programme. 

In [1]:
import os
import sys
import json
import random
import collections
import numpy as np

## Réglages de l'entrainement
On peut modifier le nombre de combats à éffectuer : 


In [2]:
# Nombre de combats 
nbFight = 30

# Remise à 0 de la table de RL 
tabRL = np.zeros(1500)

## Entrainement

### Chargement du scenario
On récupères des infromations du scenario et on modifie la valeur "random_seed" pour l'aléatoire. 

On peut à présent lancer un entrainement : 

In [3]:
for i in range(nbFight) : 
    with open("test/scenario/scenarioTrain2.json") as jsonFile:
        scenario = json.load(jsonFile)
        jsonFile.close()

    scenario["random_seed"] = random.randint(0,10000000)

    with open('test/scenario/scenarioTrain2.json', 'w') as f:
        json.dump(scenario, f)
    
    fileName = 'resultat.json'
    
    os.system('java -jar generator.jar test/scenario/scenarioTrain2.json > ' + fileName)
    
    
    with open(fileName, "r") as f:
        lines = f.readlines()
    with open(fileName, "w") as f:
        for line in lines:
            if line.strip("\n") != "db_resolver false folder=0 farmer=0":
                f.write(line)
     
    with open(fileName) as jsonFile:
        resultat = json.load(jsonFile)
        jsonFile.close()
        
        
    logs = resultat['logs']['0']

    sortedLogs = collections.OrderedDict(sorted(logs.items()))

    winnerID = int(resultat['winner']) 
    winnerName = scenario["entities"][winnerID][0]["name"]
    duration = resultat["duration"]
    execution_time = resultat["execution_time"]

    print("Combat " + str(i + 1) + " : Winner -> " + winnerName + " // Durée : " + str(duration) + " tours // Temps : " +  str(execution_time / 10000000) + " sec")
    
    for log in sortedLogs: 
        rl = str(sortedLogs[log][0][2])
        rl = rl.split('/')
        tabRL[int(rl[0])] += int(rl[1])
        tabRL[int(rl[2])] += int(rl[3])


Combat 1 : Winner -> IA // Durée : 9 tours // Temps : 0.833996 sec
Combat 2 : Winner -> IA // Durée : 12 tours // Temps : 0.9230959 sec
Combat 3 : Winner -> IA // Durée : 10 tours // Temps : 0.7577208 sec
Combat 4 : Winner -> IA // Durée : 8 tours // Temps : 0.7112666 sec
Combat 5 : Winner -> IA // Durée : 6 tours // Temps : 0.7465333 sec
Combat 6 : Winner -> IA // Durée : 8 tours // Temps : 0.7698042 sec
Combat 7 : Winner -> IA // Durée : 8 tours // Temps : 0.8150919 sec
Combat 8 : Winner -> IA // Durée : 9 tours // Temps : 0.7590834 sec
Combat 9 : Winner -> IA // Durée : 8 tours // Temps : 0.687279 sec
Combat 10 : Winner -> IA // Durée : 14 tours // Temps : 0.8794752 sec
Combat 11 : Winner -> IA // Durée : 10 tours // Temps : 0.8787873 sec
Combat 12 : Winner -> IA // Durée : 7 tours // Temps : 0.5729835 sec
Combat 13 : Winner -> IA // Durée : 8 tours // Temps : 0.7885834 sec
Combat 14 : Winner -> IA // Durée : 6 tours // Temps : 0.6962916 sec
Combat 15 : Winner -> IA // Durée : 12 to

In [4]:
print("// Tableau de renforcement")
print("global tab = [];")
for i in range(600): 
    print("tab[" + str(i) + "] = " + str(int(tabRL[i])) + ";")

// Tableau de renforcement
global tab = [];
tab[0] = 253;
tab[1] = 565;
tab[2] = 877;
tab[3] = 942;
tab[4] = 0;
tab[5] = 183;
tab[6] = 298;
tab[7] = 0;
tab[8] = 408;
tab[9] = 181;
tab[10] = 273;
tab[11] = 293;
tab[12] = 165;
tab[13] = 295;
tab[14] = 107;
tab[15] = 0;
tab[16] = 46;
tab[17] = 84;
tab[18] = 149;
tab[19] = 0;
tab[20] = 0;
tab[21] = 19;
tab[22] = 0;
tab[23] = 0;
tab[24] = 0;
tab[25] = 0;
tab[26] = 154;
tab[27] = 0;
tab[28] = 39;
tab[29] = 41;
tab[30] = 37;
tab[31] = 0;
tab[32] = 4;
tab[33] = 0;
tab[34] = 141;
tab[35] = 0;
tab[36] = 0;
tab[37] = 0;
tab[38] = 0;
tab[39] = 0;
tab[40] = 24;
tab[41] = 0;
tab[42] = 0;
tab[43] = 0;
tab[44] = 30;
tab[45] = 20;
tab[46] = 0;
tab[47] = 0;
tab[48] = 45;
tab[49] = 0;
tab[50] = 69;
tab[51] = 150;
tab[52] = 0;
tab[53] = 20;
tab[54] = 0;
tab[55] = 0;
tab[56] = 93;
tab[57] = 0;
tab[58] = 22;
tab[59] = 41;
tab[60] = 75;
tab[61] = 0;
tab[62] = 38;
tab[63] = 0;
tab[64] = 0;
tab[65] = 0;
tab[66] = 146;
tab[67] = 0;
tab[68] = 0;
tab[69] = 0;
tab

In [5]:
print(tabRL)

[253. 565. 877. ...   0.   0.   0.]


# Heuristique

Objectif : il ya encore beaucoup de 0 dans la table de RL donc on va ponderer manuellement la table avec des valeurs pour accélérer le combat. 

In [6]:
for i in range(0, 600, 16): 
    TPnull = True
    MPnull = True
    
    for tp in range(i, i+8): 
        if tabRL[tp] != 0: 
            TPnull = False
    for mp in range(i+8, i+16): 
        if tabRL[mp] != 0: 
            MPnull = False
            
    if TPnull : 
        tabRL[i+4] = 1
    if MPnull : 
        tabRL[i+10] = -1